In [12]:
import json
from pathlib import Path

import pandas as pd

In [13]:
from adapt.dvc import load_experiments

filepaths = list(Path("../../tmp/erx/").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
experiments[0]

12 experiments


{'commit': 'workspace',
 'id': 'workspace',
 'name': None,
 'params': {'task': 'erx',
  'train': {'dataset': {'path': 'bdsaglam/web_nlg-erx-concat',
    'name': 'release_v3.0_en',
    'split': 'train[:100]'},
   'optimizer': 'noop',
   'ensemble': 'no'},
  'evaluation': {'dataset': {'path': 'bdsaglam/web_nlg-erx-concat',
    'name': 'release_v3.0_en',
    'split': 'dev[:1000]'}},
  'program': {'prompting': 'structured'},
  'lm': {'model': 'llama-3-8b', 'temperature': 0.0},
  'run': 1},
 'metrics': {'exact.precision': 0.014585968840764236,
  'exact.recall': 0.013411106866989222,
  'exact.f1': 0.013893968089212146,
  'fuzzy.precision': 0.2980273675422333,
  'fuzzy.recall': 0.2736804602675239,
  'fuzzy.f1': 0.28203483319061007}}

In [14]:
df = pd.json_normalize(experiments).drop(columns=["commit"])
print(f"{len(df)} experiments before preprocessing")
df


12 experiments before preprocessing


,id,name,params.task,params.train.dataset.path,params.train.dataset.name,params.train.dataset.split,params.train.optimizer,params.train.ensemble,params.evaluation.dataset.path,params.evaluation.dataset.name,...,params.program.prompting,params.lm.model,params.lm.temperature,params.run,metrics.exact.precision,metrics.exact.recall,metrics.exact.f1,metrics.fuzzy.precision,metrics.fuzzy.recall,metrics.fuzzy.f1
0,workspace,None,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],noop,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,structured,llama-3-8b,0.0,1,0.014586,0.013411,0.013894,0.298027,0.273680,0.282035
1,58c71da3a6cb00c3bf59c04a8b3dae22395f9493,master,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],noop,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,structured,llama-3-8b,0.0,1,0.014586,0.013411,0.013894,0.298027,0.273680,0.282035
2,6f84f2616434f0cb73f78dd99349821288535df6,metal-pons,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],bfsrs-medium,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,structured,qwen-2.5-32b,0.0,1,0.377839,0.367084,0.370645,0.797541,0.779191,0.783746
3,8db0681d601021cfecaa922e7a118c7afa21a0fa,crumb-geum,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],noop,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,sft,llama-3-8b,0.0,1,0.451251,0.434675,0.437976,0.762150,0.740962,0.742936
4,914f4e695eca42802fb2d2b3ab2233cf979773a6,myoid-kefs,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],miprov2-light,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,NaN,qwen-2.5-32b,0.0,1,0.072078,0.069922,0.070684,0.532639,0.523590,0.525673
5,6d745e03f12c22dca0275b04496e83e8feb903a7,butch-keek,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],noop,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,NaN,qwen-2.5-32b,0.0,1,0.016973,0.016721,0.016769,0.447680,0.430505,0.436040
6,workspace,None,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],noop,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,NaN,llama-3-8b,0.0,1,0.014586,0.013411,0.013894,0.298027,0.273680,0.282035
7,ed74cb3591c364bf44234fe0c76b1a493846c358,alone-mesh,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],bfsrs-medium,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,NaN,qwen-2.5-32b,0.0,1,0.366102,0.358353,0.361248,0.793954,0.783383,0.786429
8,fa88c58fe75e8ecb78ffa4fdd79a9b1538ad40a7,slack-poss,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],bfsrs-medium,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,NaN,llama-3-8b,0.0,1,0.270941,0.263430,0.266274,0.686891,0.661890,0.671487
9,f939aab33f08dd257d0e54960c02affa57ff8733,group-nuke,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],miprov2-light,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,NaN,qwen-2.5-32b,0.0,1,0.064546,0.064266,0.064222,0.530839,0.524796,0.526424


In [15]:
if 'params.program.prompting' not in df.columns:
    df['params.program.prompting'] = 'structured'
df['params.program.prompting'] = df['params.program.prompting'].fillna('structured')

In [16]:
param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]
df.dropna(subset=param_cols + metric_cols, inplace=True, how="any")
df.drop_duplicates(subset=param_cols, inplace=True)

print(f"{len(df)} experiments after preprocessing")
df

11 experiments after preprocessing


,id,name,params.task,params.train.dataset.path,params.train.dataset.name,params.train.dataset.split,params.train.optimizer,params.train.ensemble,params.evaluation.dataset.path,params.evaluation.dataset.name,...,params.program.prompting,params.lm.model,params.lm.temperature,params.run,metrics.exact.precision,metrics.exact.recall,metrics.exact.f1,metrics.fuzzy.precision,metrics.fuzzy.recall,metrics.fuzzy.f1
0,workspace,None,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],noop,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,structured,llama-3-8b,0.0,1,0.014586,0.013411,0.013894,0.298027,0.273680,0.282035
2,6f84f2616434f0cb73f78dd99349821288535df6,metal-pons,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],bfsrs-medium,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,structured,qwen-2.5-32b,0.0,1,0.377839,0.367084,0.370645,0.797541,0.779191,0.783746
3,8db0681d601021cfecaa922e7a118c7afa21a0fa,crumb-geum,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],noop,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,sft,llama-3-8b,0.0,1,0.451251,0.434675,0.437976,0.762150,0.740962,0.742936
4,914f4e695eca42802fb2d2b3ab2233cf979773a6,myoid-kefs,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],miprov2-light,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,structured,qwen-2.5-32b,0.0,1,0.072078,0.069922,0.070684,0.532639,0.523590,0.525673
5,6d745e03f12c22dca0275b04496e83e8feb903a7,butch-keek,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],noop,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,structured,qwen-2.5-32b,0.0,1,0.016973,0.016721,0.016769,0.447680,0.430505,0.436040
6,workspace,None,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],noop,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,structured,llama-3-8b,0.0,1,0.014586,0.013411,0.013894,0.298027,0.273680,0.282035
7,ed74cb3591c364bf44234fe0c76b1a493846c358,alone-mesh,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],bfsrs-medium,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,structured,qwen-2.5-32b,0.0,1,0.366102,0.358353,0.361248,0.793954,0.783383,0.786429
8,fa88c58fe75e8ecb78ffa4fdd79a9b1538ad40a7,slack-poss,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],bfsrs-medium,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,structured,llama-3-8b,0.0,1,0.270941,0.263430,0.266274,0.686891,0.661890,0.671487
9,f939aab33f08dd257d0e54960c02affa57ff8733,group-nuke,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],miprov2-light,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,structured,qwen-2.5-32b,0.0,1,0.064546,0.064266,0.064222,0.530839,0.524796,0.526424
10,d82bf43700c686be11581eceae44b54b04b1299c,zippy-cane,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],miprov2-light,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,structured,llama-3-8b,0.0,1,0.133240,0.133156,0.132834,0.588515,0.593496,0.588394


In [17]:
def sorted_tuple(x):
    return tuple(sorted(x))

run_counts = (
    df.groupby(param_cols[:-1])[
        "params.run"
    ]
    .aggregate(sorted_tuple)
    .reset_index()
)

In [18]:
for col in run_counts.columns:
    print(f"- {col}")
    for value in run_counts[col].unique():
        print(f"\t{value}")

- params.task
	erx
- params.train.dataset.path
	bdsaglam/web_nlg-erx-concat
- params.train.dataset.name
	release_v3.0_en
- params.train.dataset.split
	train[:100]
- params.train.optimizer
	bfsrs-medium
	miprov2-light
	noop
- params.train.ensemble
	no
- params.evaluation.dataset.path
	bdsaglam/web_nlg-erx-concat
- params.evaluation.dataset.name
	release_v3.0_en
- params.evaluation.dataset.split
	dev[:1000]
	dev[:100]
- params.program.prompting
	structured
	sft
- params.lm.model
	qwen-2.5-32b
	llama-3-8b
- params.lm.temperature
	0.0
- params.run
	(1,)


In [19]:
df.to_json('exps.jsonl', orient='records', lines=True)

In [20]:
param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]

In [21]:
df[['name', 'params.program.prompting', 'params.train.optimizer', 'params.lm.model', 'params.lm.temperature', *metric_cols]].sort_values(by='metrics.fuzzy.f1', ascending=False)

,name,params.program.prompting,params.train.optimizer,params.lm.model,params.lm.temperature,metrics.exact.precision,metrics.exact.recall,metrics.exact.f1,metrics.fuzzy.precision,metrics.fuzzy.recall,metrics.fuzzy.f1
7,alone-mesh,structured,bfsrs-medium,qwen-2.5-32b,0.0,0.366102,0.358353,0.361248,0.793954,0.783383,0.786429
2,metal-pons,structured,bfsrs-medium,qwen-2.5-32b,0.0,0.377839,0.367084,0.370645,0.797541,0.779191,0.783746
3,crumb-geum,sft,noop,llama-3-8b,0.0,0.451251,0.434675,0.437976,0.762150,0.740962,0.742936
8,slack-poss,structured,bfsrs-medium,llama-3-8b,0.0,0.270941,0.263430,0.266274,0.686891,0.661890,0.671487
10,zippy-cane,structured,miprov2-light,llama-3-8b,0.0,0.133240,0.133156,0.132834,0.588515,0.593496,0.588394
9,group-nuke,structured,miprov2-light,qwen-2.5-32b,0.0,0.064546,0.064266,0.064222,0.530839,0.524796,0.526424
4,myoid-kefs,structured,miprov2-light,qwen-2.5-32b,0.0,0.072078,0.069922,0.070684,0.532639,0.523590,0.525673
11,sated-pond,structured,noop,qwen-2.5-32b,0.0,0.020862,0.020312,0.020562,0.455415,0.439280,0.445118
5,butch-keek,structured,noop,qwen-2.5-32b,0.0,0.016973,0.016721,0.016769,0.447680,0.430505,0.436040
0,None,structured,noop,llama-3-8b,0.0,0.014586,0.013411,0.013894,0.298027,0.273680,0.282035
